In [1]:
# CVSIM - IMPORT AND DEFINITIONS

import os
clear = lambda: os.system('clear')
clear()
#sys.path.append(os.path.dirname(os.path.abspath(__file__)))
script_dir = os.path.dirname(os.path.realpath(__file__))
os.chdir(script_dir)
import sys
import cProfile
import pstats
import numpy as np #Numpy is a 'fundamental package for scientific computing with Python'
np.set_printoptions(threshold=sys.maxsize) # This makes sure that all elements are printed, without truncation.
import scipy as sp # SciPy is a free and open-source Python library used for scientific computing and technical computing
import matplotlib.pyplot as plt # Matplotlib is a comprehensive library for creating static, animated, and interactive visualizations in Python.
import scipy.signal as signal
from scipy.io import loadmat
import timerz
import math
from scipy.integrate import solve_ivp
from scipy.interpolate import interp1d
import pandas as pd
import csv
import platform

# Load the right parameters settings
#from adultPars import _init_pars # Get the parameters for resistance, elastance and uvolume
#from controlPars import _init_control # Get the control parameters loaded.
#from reflexPars import _init_reflex # Get the control parameters loaded.

#import PySimpleGUI as sg

NameError: name '__file__' is not defined

In [ ]:
# General settings

print('Healthy adult sim run')
from adultPars_carotid import _init_pars # Get the parameters for resistance, elastance and uvolume
from controlPars import _init_control # Get the control parameters loaded.
#from reflexPars import _init_reflex # Get the control parameters loaded.
planet=1
fluidLoading=0
StStest=1
cerebralModelOn=0
if cerebralModelOn==1: # Change these
    cerebralVeinsOn=0
    carotidOn=1
else: # Do not change these
    cerebralVeinsOn=0
    carotidOn=0
ABRreflexOn=1; # ABR reflex
CPRreflexOn=1; # CPR reflex
micro_switch = 1 # 1 = microcirculation on, 0 = microcirculation off
bleedingOn,startTimeBleeding,totalTimeBleeding,bleedingVolume=0,150,180,2000;#  time in seconds
fillingOn,totalTimeFilling,fillingVolume=0,200,1000;#  time in seconds

if cerebralModelOn==1:
    print("BRAIN = ON")
    if cerebralVeinsOn==0:
        print("BRAIN VEINS = OFF")
    else:
        print("BRAIN VEINS = ON")
    if carotidOn==1:
        print("CAROTID = ON")
    else:
        print("CAROTID = OFF")
else:
    print("BRAIN = OFF")

# Get all the pars from the different parameters values
#global subjectPars,controlPars,reflexPars
subjectPars = _init_pars(); # Here the compartments parameters are assigned
controlPars=_init_control(); # Get all the control parameters stored to the list 'control'.You can access the heart rate for instance by contorl.hr.
#reflexPars=_init_reflex(); # Get all the reflex parameters stored to the list 'reflexPars'.


In [ ]:
### DEF ###
def find_index_of_time(time_array, target_time):
    indices = np.where(time_array == target_time)[0]  # Get indices where target_time occurs
    if indices.size > 0:  # Check if indices array is not empty
        return indices[0]  # Return the first index where the target_time occurs
    else:
        print(f"Time value {target_time} not found in the array.")
        return None
    
def round_time_series(time_series, decimals=4):
    return [round(element, decimals) for element in time_series]

def UV_lim(compartment, comp_norm):
    if compartment < 1/limit*comp_norm:
        compartment = 1/limit*comp_norm
        #print("UV lower limit")
    if compartment > limit*comp_norm:
        compartment = limit*comp_norm
        #print("UV upper limit")
    return compartment
        
def R_lim(compartment, comp_norm):
    if compartment < 1/limit*comp_norm:
        compartment = 1/limit*comp_norm
        #print("R lower limit")
    if compartment > limit*comp_norm:
        compartment = limit*comp_norm
        #print("R upper limit")
    return compartment

def get_data(filename):
    global data, data_DBP, data_SBP, data_time, data_BP, data_time_10Hz, data_MAP
    # Load .mat file
    data = loadmat(filename)
    data_time = np.squeeze(data['data']['time'][0][0])
    data_BP = np.squeeze(data['data']['BP'][0][0])
    data_time_10Hz = np.squeeze(data['data']['time_10Hz'][0][0])
    data_MAP = np.squeeze(data['data']['map'][0][0])
    data_SBP = np.squeeze(data['data']['sbp'][0][0])
    data_DBP = np.squeeze(data['data']['dbp'][0][0])
    
def extract_patient_id(filename):
    # Extract the patient ID using string manipulation
    # Split the path into parts and find the part that contains 'PHI'
    parts = filename.split(os.sep)
    patient_id = None
    for part in parts:
        if 'PHI' in part:
            patient_id = part
            break
    
    # Extracted patient ID
    if patient_id:
        print(f'Extracted patient ID: {patient_id}')
        return patient_id
    else:
        print('Patient ID not found in the given file path.')

def alpha_rad_full_func(tilt_time, tau_STS, p=0.25):
    """
    Calculate alpha_rad_full with flattened maxima and constant peak value.
    
    Parameters:
    tilt_time (array-like): Time values for the tilt motion.
    tau_STS (float): Period of the tilt motion.
    p (float): Power to flatten the maxima (p >= 1).
    
    Returns:
    numpy.ndarray: Computed alpha_rad_full values.
    """
    cos_term = 1.0 - np.cos(2 * np.pi * tilt_time / tau_STS)
    normalization = (1.0 - np.cos(np.pi)) ** p
    return cos_term ** p / normalization
      
        